In [3]:
import torch
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import Dataset
import sacrebleu

In [2]:
def read_binary_file(file_path):
    with open(file_path, 'rb') as file:
        lines = file.read().decode('utf-8').split('\n')
    return lines


gujarati_text = read_binary_file('train_datasets/devtest.guj_Gujr')
nepali_text = read_binary_file('train_datasets/devtest.npi_Deva')
burmese_text = read_binary_file('train_datasets/devtest.mya_Mymr')
khmer_text = read_binary_file('train_datasets/devtest.khm_Khmr')
galician_text = read_binary_file('train_datasets/devtest.glg_Latn')
english_labels = read_binary_file('train_datasets/devtest.eng_Latn')

In [ ]:
model_name = 'facebook/mbart-large-50-many-to-one-mmt'
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)

def encode(dataset, max_length=128):
    inputs = tokenizer(dataset['source'], padding="max_length", truncation=True, max_length=max_length)
    outputs = tokenizer(dataset['target'], padding="max_length", truncation=True, max_length=max_length)
    return {"input_ids": inputs.input_ids, "attention_mask": inputs.attention_mask,
            "labels": outputs.input_ids}

# Create a dataset object

# gujarati_dataset = Dataset.from_dict({'source': gujarati_text, 'target': english_labels})
# nepali_dataset = Dataset.from_dict({'source': nepali_text, 'target': english_labels})
# burmese_dataset = Dataset.from_dict({'source': burmese_text, 'target': english_labels})
# khmer_dataset = Dataset.from_dict({'source': khmer_text, 'target': english_labels})
# galician_dataset = Dataset.from_dict({'source': galician_text, 'target': english_labels})

# encoded_gujarati = [encode("gu_IN", i) for i in gujarati_dataset]
# encoded_nepali = [encode("ne_NP", i) for i in nepali_dataset]
# encoded_burmese = [encode("my_MM", i) for i in burmese_dataset]
# encoded_khmer = [encode("km_KH", i) for i in khmer_dataset]
# encoded_galician = [encode("gl_ES", i) for i in galician_dataset]

In [15]:
# model_name = 'facebook/mbart-large-50-many-to-one-mmt'
# tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
# tokenizer.src_lang = "gu_IN"  # Setting the source language

# from transformers import DataCollatorForSeq2Seq

# class CustomDataCollator(DataCollatorForSeq2Seq):
#     def __init__(self, tokenizer, model=None, padding=True, max_length=None):
#         super().__init__(tokenizer, model=model, padding=padding, max_length=max_length)

#     def __call__(self, features):
#         labels = [example['target'] for example in features]  # Gather target texts
#         model_inputs = self.tokenizer([example['source'] for example in features], 
#                                       max_length=self.max_length, 
#                                       padding=self.padding, 
#                                       truncation=True)
#         with self.tokenizer.as_target_tokenizer():
#             labels = self.tokenizer(labels, max_length=self.max_length, padding=self.padding, truncation=True)

#         labels["input_ids"] = [
#             [(label if label != self.tokenizer.pad_token_id else -100) for label in labels_example]
#             for labels_example in labels["input_ids"]
#         ]

#         model_inputs["labels"] = labels["input_ids"]
#         return model_inputs



# gujarati_dataset = Dataset.from_dict({'source': gujarati_text, 'target': english_labels})

# model = MBartForConditionalGeneration.from_pretrained(model_name).to('cuda')

# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     per_device_train_batch_size=8,  
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     fp16=True,
#     gradient_accumulation_steps=4
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=gujarati_dataset,
#     data_collator=CustomDataCollator(tokenizer, model, padding="max_length", max_length=128)
# )

# trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 0 has a total capacty of 10.75 GiB of which 20.50 MiB is free. Process 90784 has 10.73 GiB memory in use. Of the allocated memory 10.54 GiB is allocated by PyTorch, and 43.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Gujarati

In [7]:
tokenizer.src_lang = "gu_IN"

gujarati_dataset = Dataset.from_dict({'source': gujarati_text, 'target': english_labels})
encoded_gujarati = gujarati_dataset.map(encode, batched=True)

model = MBartForConditionalGeneration.from_pretrained(model_name).to('cuda') 

training_args = TrainingArguments(
    output_dir='./results',         
    evaluation_strategy="epoch",    
    learning_rate=2e-5,
    weight_decay=0.01,             
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,       
    num_train_epochs=3, 
    fp16=True,
    gradient_accumulation_steps=4 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_gujarati,
    tokenizer = tokenizer
)

trainer.train()

Map:   0%|          | 0/1013 [00:00<?, ? examples/s]

/home/yiy033/.local/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 10.75 GiB of which 6.50 MiB is free. Process 86301 has 10.74 GiB memory in use. Of the allocated memory 10.53 GiB is allocated by PyTorch, and 23.82 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF